In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hypertension/GSE205138'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"miRNA expression in tracheal aspirates [sBPD, BPD-PH]"
!Series_summary	"Extreme preterm infants are a growing population in the neonatal intensive care unit.  Multiple factors play a role in preterm birth, resulting in complications including severe bronchopulmonary dysplasia (sBPD) without or with and pulmonary hypertension (BPD-PH).  The goal of this study was to identify biomarker signatures associated with sBPD and BPD-PH.  We analyzed profiles in tracheal aspirates (TAs) from 46 extremely preterm infants receiving invasive mechanical ventilation (25 sBPD, 21 BPD-PH) . We found specific miRNA signatures in TAs that may serve as biomarkers for the two disease phenotypes."
!Series_overall_design	"We extracted RNA  from 0.5 mL of tracheal aspirates from premature neonates with BPD and BPD-PH using the Plasma/Serum RNA Purification Kit (Norgen). The expression of 1,066 human miRNAs was quantified using the Human miRNome miScript® miRNA PCR Array (

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Gene Expression Data Availability
if 'gene expression' in ['miRNA expression']:  # dataset description suggests it's miRNA data
    is_gene_available = False
else:
    is_gene_available = True

# Variable Availability and Data Type Conversion
sample_characteristics = {
    0: ['developmental stage: premature neonate'],
    1: ['tissue: tracheal aspirate'],
    2: ['sbpd: no', 'sbpd: yes'],
    3: ['bpd-ph: yes', 'bpd-ph: no']
}

# Checking for Hypertension (trait)
if 3 in sample_characteristics and len(set(value.split(': ')[1] for value in sample_characteristics[3])) > 1:
    trait_row = 3

# Age and Gender are not available explicitly in the sample characteristics dictionary
age_row = None
gender_row = None

# Data Type Conversion Functions
def convert_trait(value):
    if value:
        value = value.split(': ')[1].strip().lower()
        return 1 if value == 'yes' else 0 if value == 'no' else None
    return None

convert_age = lambda x: None  # since age is not available
convert_gender = lambda x: None  # since gender is not available

# Save Metadata
save_cohort_info('GSE205138', './preprocessed/Hypertension/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Hypertension', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Hypertension/trait_data/GSE205138.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM6205639': [1], 'GSM6205640': [0], 'GSM6205641': [0], 'GSM6205642': [1], 'GSM6205643': [1], 'GSM6205644': [1], 'GSM6205645': [0], 'GSM6205646': [0], 'GSM6205647': [0], 'GSM6205648': [1], 'GSM6205649': [1], 'GSM6205650': [0], 'GSM6205651': [0], 'GSM6205652': [1], 'GSM6205653': [1], 'GSM6205654': [0], 'GSM6205655': [1], 'GSM6205656': [0], 'GSM6205657': [0], 'GSM6205658': [1], 'GSM6205659': [1], 'GSM6205660': [0], 'GSM6205661': [0], 'GSM6205662': [0], 'GSM6205663': [0], 'GSM6205664': [1], 'GSM6205665': [0], 'GSM6205666': [1], 'GSM6205667': [0], 'GSM6205668': [0], 'GSM6205669': [1], 'GSM6205670': [1], 'GSM6205671': [1], 'GSM6205672': [1], 'GSM6205673': [0], 'GSM6205674': [1], 'GSM6205675': [0], 'GSM6205676': [1], 'GSM6205677': [0], 'GSM6205678': [0], 'GSM6205679': [1], 'GSM6205680': [0], 'GSM6205681': [1], 'GSM6205682': [0], 'GSM6205683': [0], 'GSM6205684': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['hsa-let-7a', 'hsa-let-7b', 'hsa-let-7c', 'hsa-miR-1183',
       'hsa-miR-1203', 'hsa-miR-1207-5p', 'hsa-miR-1224-5p', 'hsa-miR-1233',
       'hsa-miR-1237', 'hsa-miR-1238', 'hsa-miR-1247', 'hsa-miR-1260',
       'hsa-miR-127-3p', 'hsa-miR-1280', 'hsa-miR-1281', 'hsa-miR-1285',
       'hsa-miR-1287', 'hsa-miR-1290', 'hsa-miR-1301', 'hsa-miR-1303'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# requires_gene_mapping is False, skip STEP6.


Gene annotation preview:
{'ID': ['hsa-miR-346', 'hsa-miR-196b', 'hsa-miR-223', 'hsa-miR-608', 'hsa-miR-211'], 'SPOT_ID': ['hsa-miR-346', 'hsa-miR-196b', 'hsa-miR-223', 'hsa-miR-608', 'hsa-miR-211']}


### Step 7: Data Normalization and Merging

In [7]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Hypertension/gene_data/GSE205138.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Hypertension')

# 4. Save the cohort information.
save_cohort_info('GSE205138', './preprocessed/Hypertension/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Hypertension/GSE205138.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
